<div class="alert alert-block alert-warning">
<b>Prerequisites:</b> Please run the prerequiresites <b>00-prerequisites.ipynb</b> first before proceeding.
</div>

## Introduction to image-to-video

With Nova Reel's image-to-video capability, you can create a video from an existing image, giving you greater control over your video output that you get with text-to-video alone. The image you use could be a real world photograph, an image created by an artist or designer, a rendering of a real product, or an image generated with an image generation model.

Run the cell below to create an instance of the Bedrock Runtime client which we'll use later.


In [1]:
import boto3
import video_gen_util

boto3.setup_default_session(region_name="us-east-1")
session = boto3.Session()
sts_client = session.client("sts")

bedrock_runtime = boto3.client("bedrock-runtime")

#### Setting up storage

Generating a video takes some time - approximately 3-4 minutes to produce a 6 second video. To accomodate this execution time, the Bedrock Runtime introduces a new asynchronous invocation API. You start generating a video by calling the `start_async_invoke()` method on the Bedrock Runtime client. When the generation job completes, Bedrock automatically saves the generated video to an S3 bucket you specify.

Run the cell below to automatically create a new S3 bucket in your account which we will use as a destination for your videos. (If you already have an existing bucket you'd like to use, edit the `s3_destination_bucket` variable to use that bucket's ID)


In [2]:
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]


create_new_bucket = False

if create_new_bucket == True:
    new_bucket_name = f"ovg-bucket-{region}-{account_id}"
    s3_destination_bucket = new_bucket_name
    # Create the bucket
    boto3.client("s3").create_bucket(Bucket=s3_destination_bucket)
else:
    # Replace this with an existing bucket ID if you'd like.
    s3_destination_bucket = 'sagemaker-us-west-2-710299592439/genai/reel/'

#### Create prompt and invoke model

OctankFashion would like to take the marketing image you previously created and bring it to life as a video. Nova Reel's image-to-video features is the perfect fit.

The following parameters are relevant to using the image-to-video feature. The parameters are encapsulated in the `textToVideoParams` field of the request body.

- `text` (Optional) – A text prompt describing your desired video. Must be 1 - 512 characters in length.
- `images` (Optional) - A list containing exactly one image source.

The image source structure should look like this:

```
{
    "format": "png" | "jpeg"
    "source": {
        "bytes": string (Base64 encoded image)
    }
}
```

Here is the image you'll be using as the basis for the video. Images that are used as input for Nova Reel must be 1280x720, the same resolution that the generated video will be. For best results, the text prompt you use should describe the image in full along with any action or camera motion that you would like to include in the video.

Run the cells below to start generating your video.

<img src="data/tshirt_beach_1280x720.png" height="400">


In [3]:
# Define the main input parameters.
input_image_path = "data/tshirt_beach_1280x720.png"  # Must be 1280 x 720
image_format = "png"  # Can be "png" or "jpeg"

text = "static camera: waves roll in, sandy beach, a man wearing a tshirt with a palm tree graphic on it. The man shifts his body subtley"

seed = 546416566

In [4]:
import json
import base64

# Load the input image as a Base64 string.
with open(input_image_path, "rb") as f:
    input_image_bytes = f.read()
    input_image_base64 = base64.b64encode(input_image_bytes).decode("utf-8")

model_input = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {
        "text": text,
        "images": [{"format": image_format, "source": {"bytes": input_image_base64}}],
    },
    "videoGenerationConfig": {
        "durationSeconds": 6,  # 6 is the only supported value currently.
        "fps": 24,  # 24 is the only supported value currently.
        "dimension": "1280x720",  # "1280x720" is the only supported value currently.
        "seed": seed,  # Can be any random number between 0 to 2147483648
    },
}

# Start the asynchronous video generation job.
invocation = bedrock_runtime.start_async_invoke(
    modelId="amazon.nova-reel-v1:0",
    modelInput=model_input,
    outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{s3_destination_bucket}"}},
)

# Pretty print the response JSON.
print("\nResponse:")
print(json.dumps(invocation, indent=2, default=str))

# Save the invocation details for later reference. Helpful for debugging and reporting feedback.
video_gen_util.save_invocation_info(invocation, model_input)


Response:
{
  "ResponseMetadata": {
    "RequestId": "75f8c99f-58c0-4286-860d-2c80d8e5de60",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 07 Feb 2025 09:44:46 GMT",
      "content-type": "application/json",
      "content-length": "84",
      "connection": "keep-alive",
      "x-amzn-requestid": "75f8c99f-58c0-4286-860d-2c80d8e5de60"
    },
    "RetryAttempts": 0
  },
  "invocationArn": "arn:aws:bedrock:us-east-1:710299592439:async-invoke/1batsl94ss4h"
}


'/home/sagemaker-user/amazon-nova-samples/multimodal-generation/workshop-sample/NovaReel/output/2025-02-07_09-44-45_1batsl94ss4h'

In [7]:
import time
# Check CMI job status
invocation_arn = invocation["invocationArn"]
while True:
    response = bedrock_runtime.get_async_invoke(invocationArn=invocation_arn)
    status = response["status"]
    print(f"Status: {status}")
    
    if status in ['Completed', 'Failed']:
        break
        
    time.sleep(60)  # Check every 60 seconds

Status: InProgress
Status: InProgress
Status: Completed


In [9]:
import boto3
from botocore.exceptions import ClientError
from urllib.parse import urlparse

def parse_s3_uri_with_urllib(s3_uri):
    """
    Parse an S3 URI using urllib
    
    Args:
        s3_uri (str): S3 URI in format 's3://bucket-name/prefix/path'
        
    Returns:
        tuple: (bucket_name, prefix)
    """
    parsed = urlparse(s3_uri)
    bucket_name = parsed.netloc
    prefix = parsed.path.lstrip('/')
    
    return bucket_name, prefix
    
def download_file(s3Uri, file_name, local_path):
    """
    Download a file from S3 using prefix/key
    """
    s3_client = boto3.client('s3')
    bucket_name, prefix = parse_s3_uri_with_urllib(s3Uri)
    print(f"\nURI: {s3Uri}, Bucket: {bucket_name}, Prefix: {prefix}")
    try:
        s3_client.download_file(
            Bucket=bucket_name,
            Key=f"{prefix}/{file_name}",
            Filename=local_path
        )
        print(f"Successfully downloaded {file_name} to {local_path}")
    except ClientError as e:
        print(f"Error downloading file: {e}")
        return False
    return True

In [11]:
import os

completed_job = bedrock_runtime.get_async_invoke(invocationArn=invocation_arn)

job_id = video_gen_util.get_job_id_from_arn(job["invocationArn"])
print(f"try to download completed job {job_id}.")
output_s3_uri = (
    job["outputDataConfig"]["s3OutputDataConfig"]["s3Uri"]
)
print(f"The asyncInvokeSummaries s3Uri {output_s3_uri}")
#video_gen_util.save_completed_job(job)
output_folder = "ouput"
output_folder_abs = os.path.abspath(
    f"{output_folder}/{video_gen_util.get_folder_name_for_job(job)}"
)
# Ensure the output folder exists
os.makedirs(output_folder_abs, exist_ok=True)
output_filename = "output.mp4"
local_file_path = os.path.join(output_folder_abs, output_filename)
print(f"save file to {local_file_path}")
    
download_file(output_s3_uri, output_filename, local_file_path)
    

try to download completed job g2rd27o9bikk.
The asyncInvokeSummaries s3Uri s3://sagemaker-us-west-2-710299592439/genai/reel/g2rd27o9bikk
save file to /home/sagemaker-user/amazon-nova-samples/multimodal-generation/workshop-sample/NovaReel/ouput/2025-02-07_07-39-32_g2rd27o9bikk/output.mp4

URI: s3://sagemaker-us-west-2-710299592439/genai/reel/g2rd27o9bikk, Bucket: sagemaker-us-west-2-710299592439, Prefix: genai/reel/g2rd27o9bikk
Successfully downloaded output.mp4 to /home/sagemaker-user/amazon-nova-samples/multimodal-generation/workshop-sample/NovaReel/ouput/2025-02-07_07-39-32_g2rd27o9bikk/output.mp4


True

##### Download and view the generated videos

We've provided a set of utility functions in the `video_gen_util.py` script. One of these functions provides one solution to automatically downloading previously completed jobs and monitoring in-progress jobs. Finished jobs will be automatically downloaded to the "output" folder.


In [8]:
from datetime import datetime, timedelta, timezone

# Download and monitor videos from the past N hours.
duration_hours = 2

from_submit_time = datetime.now(timezone.utc) - timedelta(hours=duration_hours)
video_gen_util.monitor_and_download_videos("output", submit_time_after=from_submit_time)

Problem: No MP4 file was found in S3 at sagemaker-us-west-2-710299592439/1batsl94ss4h
Monitoring 0 "InProgress" jobs.
Monitoring and download complete!


Now, check the `output` folder in your Sagemaker, and you can view the video files in `.mp4` format.


## Take Away

Amazon Nova Reel's image-to-video feature gives you greater control over the subject, layout, and visual aesthetic of your generated video. It's a perfect fit when you need to create a video featuring a real world product or subject.
